In [1]:
import numpy as np

In [2]:
%ls

 cov_inv_test                  params
 cov_invv                      params_test
 data_read.py                  pk99.h5
 dT_b                          pk_test
 EMuPk222.h5                   powerspectrum
 history.npy                   Powerspectrum_THANN_burnin.out
 index                         Powerspectrum_THANN_burninprob.out
 k.txt                         Powerspectrum_THANN_.log
 nbins_test                    Powerspectrum_THANN_.out
 neutral_frac                  Powerspectrum_THANN_prob.out
 nkbins                        script.py
 norm_pk                       training.ipynb
'parameter estimation.ipynb'   Untitled1.ipynb


In [3]:
from tensorflow import keras as ks

model = ks.models.load_model('EMuPk222.h5')
def pk_th(params):
    
    
    Pk_pdct = model.predict([[params]])
    
    return Pk_pdct

In [4]:
pk = np.loadtxt('pk_test')
params = np.loadtxt('params_test')
nbins = np.loadtxt('nkbins')
pk.shape, params.shape

((18, 7), (18, 3))

In [5]:
ind = np.loadtxt('index')
index = np.zeros(shape=len(ind),dtype=int)
for i in range(len(index)):
    index[i] = int(ind[i])
    

In [6]:
nbins = nbins[index]

In [7]:
params

array([[  20.  ,   20.  ,  500.  ],
       [  55.  ,   10.  ,  100.  ],
       [ 130.  ,   30.  ,  100.  ],
       [  40.  ,   25.  ,  200.  ],
       [  23.21,   20.  , 1000.  ],
       [  45.  ,   60.  ,  700.  ],
       [  40.  ,   30.  ,  500.  ],
       [  20.  ,   25.  ,   10.  ],
       [  30.  ,   10.  ,   10.  ],
       [ 180.  ,   20.  ,   50.  ],
       [ 140.  ,   30.  ,  100.  ],
       [ 100.  ,   10.  ,  100.  ],
       [ 130.  ,   45.  ,  200.  ],
       [  50.  ,   60.  ,  800.  ],
       [  18.  ,   45.  ,  100.  ],
       [ 140.  ,   10.  ,   10.  ],
       [ 170.  ,   30.  ,  800.  ],
       [ 200.  ,   30.  ,  800.  ]])

In [8]:
nbins.shape, pk.shape,len(pk)

((18, 7), (18, 7), 18)

In [9]:
def cov(ind):
    cov = np.zeros(shape=(len(nbins),7,7))
    for i in range(len(nbins)):
        for j in range(7):
            cov[i,j,j] = pk[i,j]/np.sqrt(nbins[i][j])
        cov[i]=np.linalg.inv(cov[i])
    return cov[ind]

In [10]:
cov(0)

array([[ 2.1753372 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.26932305,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  3.02919442,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  4.13308256,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  5.74816308,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         8.67780566,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , 15.75107179]])

In [11]:
def logL(params):
    
    nion , rmfp , noh = params
    
    if nion < 0. or nion > 200. or rmfp < 0. or rmfp > 130. or noh < 50. or noh > 1500.:
        logl = -np.inf
        
    else:
        ind = 12
        pwk = np.reshape(pk[ind],(1,7))
        diff = np.subtract(pwk,pk_th(params))
        #diff = diff/1000.
        cov_inv = cov(ind)
        

        logl = -0.5*np.dot(np.dot(diff,cov_inv),diff.T)
        logl = logl[0][0]
        
    return logl
        

In [12]:
nwalkers = 6
ndim = 3

p  = 200*np.random.rand(nwalkers,ndim,)
p  = np.around(p,1)
p

array([[ 12.4,   7.3, 110.2],
       [ 94.6,  98.3, 123.7],
       [ 88. , 125.7, 135.8],
       [ 85. , 141.9, 149.7],
       [114. , 152. , 200. ],
       [ 17.9,  71.4,  78. ]])

In [17]:
logL(p[2])

-442.00385931408766

In [18]:
import emcee 
from multiprocessing import Pool
import time
#with Pool() as pool:


In [19]:


sampler = emcee.EnsembleSampler(
    nwalkers,
    ndim,
    logL,
    moves=[(emcee.moves.DEMove(), 1.5), (emcee.moves.DESnookerMove(), 2.0),],
threads=16)
sampler.run_mcmc(p, 5000)

emcee: Exception while calling your likelihood function:
  params: [123.40633381  88.29339692 152.14767652]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/ht/.local/lib/python3.6/site-packages/emcee/ensemble.py", line 495, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-11-16d25da66049>", line 11, in logL
    diff = np.subtract(pwk,pk_th(params))
  File "<ipython-input-3-33509861a92e>", line 7, in pk_th
    Pk_pdct = model.predict([[params]])
  File "/home/ht/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 1013, in predict
    use_multiprocessing=use_multiprocessing)
  File "/home/ht/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 498, in predict
    workers=workers, use_multiprocessing=use_multiprocessing, **kwargs)
  File "/home/ht/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 426, in _model_iteration
    use_multiprocessing=use_multiprocessing)
  File "/home/ht/.local/lib/python3.6/site-packages/tensorflow_core/python/ke

KeyboardInterrupt: 

In [ ]:
s_flat = sampler.flatchain
logl  = sampler.flatlnprobability

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(s_flat[:,1],s_flat[:,2],c = -logl)
plt.colorbar()

In [ ]:
plt.subplot(1,3,1)
plt.hist(s_flat[:,0],bins=50)
plt.subplot(1,3,2)
plt.hist(s_flat[:,1],bins=50)
plt.subplot(1,3,3)
plt.hist(s_flat[:,2],bins=50)
params[2]

In [ ]:
import os

os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
import emcee 
from multiprocessing import Pool
import time
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers,ndim,logL,blobs_dtype=float,pool=pool)
    start = time.time()
    sampler.run_mcmc(p,100,progress= True)
    s_flat = sampler.flatchain
    logp = sampler.lnprobability
    log_prior_samps = sampler.get_blobs()
    flat_log_prior_samps = sampler.get_blobs(flat=True)
    end = time.time()
    t = end -start
print('%f sec'%t)

In [ ]:
from multiprocessing import cpu_count
ncpu = cpu_count()
print("{0} CPUs".format(ncpu))

In [ ]:
with open("script.py", "w") as f:
    f.write("""
import sys
import time
import emcee
import numpy as np
from schwimmbad import MPIPool

pk = np.loadtxt('pk_test')
cov = np.loadtxt('cov_inv_test')
params = np.loadtxt('params_test')

def logL(params):
    
    nion , rmfp , noh = params
    
    if nion < 0 or nion > 200 or rmfp < 0 or rmfp > 150 or noh < 30 or noh > 1500:
        logl = -np.inf
        
    else:
        diff = pk[0]-pk_th(params)
        
        logl = -0.5*np.dot(np.dot(diff,cov),diff.T)
        
    return logl
        

with MPIPool() as pool:
    if not pool.is_master():
        pool.wait()
        sys.exit(0)

    nwalkers = 6
    ndim = 3

    p  = 100*np.random.rand(nwalkers,ndim)
    
    nsteps = 1000

    sampler = emcee.EnsembleSampler(nwalkers, ndim, logL, pool=pool)
    start = time.time()
    sampler.run_mcmc(p, nsteps)
    s_flat = sampler.flatchain
    logp = sampler.lnprobability
    end = time.time()
    print(end - start)
    np.savetxt('sflat',s_flat)
    np.savetxt('logp',logp)
    
""")

mpi_time = !mpiexec -n {ncpu} python script.py
#mpi_time = float(mpi_time[0])
#print("MPI took {0:.1f} seconds".format(mpi_time))
#print("{0:.1f} times faster than serial".format(serial_time / mpi_time))

In [ ]:
%ls